In [ ]:
import org.apache.spark.streaming.StreamingContext._
import org.apache.spark.streaming.{Seconds, StreamingContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.SQLContext
import org.apache.spark.streaming.Duration
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.types
import spark.implicits._

In [ ]:
!rm -r checkpoint


In [ ]:
!mkdir checkpoint

In [ ]:
spark.sql("set spark.sql.catalogImplementation").show()

In [ ]:
val df = spark.read
    .json("input/*.json")

In [ ]:
df.count()

In [ ]:
val schema = df.schema

In [ ]:
val df = spark.readStream
    .format("json")
    .schema(schema)
    .option("multiLine", "false")
    .option("cleanSource", "archive")
    .option("sourceArchiveDir", f"input_archive")
    .load("input/*.json")

In [ ]:
val df2 = df.writeStream
    .option("checkpointLocation", "checkpoint")
    .trigger(org.apache.spark.sql.streaming.Trigger.Once)
    .toTable("houses")

In [ ]:
df2.processAllAvailable
//df2.stop
df2.awaitTermination

In [ ]:
val df3 = spark.read.format("jdbc").table("houses")

In [ ]:
df3.count

In [ ]:
df2.stop

In [ ]:
df3.show()